In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HP\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Text-Summarizer'

In [5]:
#update entity - return type of function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

    

In [6]:
#update config manager
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL = config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir
          )

          return data_ingestion_config

In [14]:
# update the components
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
import logging

In [25]:
import urllib.request 


class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        source_url = self.config.source_URL
        local_file_path = self.config.local_data_file
        urllib.request.urlretrieve(source_url, local_file_path)
        logging.info(f"Downloaded file from {source_url} to {local_file_path}")

        # Log the file size and type
        if os.path.exists(local_file_path):
            logging.info(f"File size: {os.path.getsize(local_file_path)} bytes")
            with open(local_file_path, 'rb') as file:
                header = file.read(4)
                logging.info(f"File header: {header}")

    def extract_zip_file(self):
        local_file_path = self.config.local_data_file
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(local_file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logging.info(f"Extracted {local_file_path} to {unzip_path}")
        except zipfile.BadZipFile:
            logging.error(f"File {local_file_path} is not a zip file")


In [27]:
#create the pipeline
try:
    config =  ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-16 22:50:16,483: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-16 22:50:16,485: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-16 22:50:16,486: INFO: common: created directory at: artifacts]
[2024-07-16 22:50:16,489: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-16 22:50:18,632: INFO: 504335435: Downloaded file from https://raw.githubusercontent.com/aasawari5/Text-Summarizer/main/summarizer-data.zip to artifacts/data_ingestion_data.zip]
[2024-07-16 22:50:18,633: INFO: 504335435: File size: 7903594 bytes]
[2024-07-16 22:50:18,633: INFO: 504335435: File header: b'PK\x03\x04']
[2024-07-16 22:50:18,739: INFO: 504335435: Extracted artifacts/data_ingestion_data.zip to artifacts/data_ingestion]
